# <center> Data Preprocessing </center> 

In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from pymystem3 import Mystem
from tqdm import tqdm_notebook

import sys
sys.path.append('../')
from src.utils import remove_name_from_text

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
data = pd.read_pickle('../data/raw/transcripts_1994_2021.pkl')

In [3]:
data.head()

,start_line,end_line,type,lines,votes,deputy_name,meeting_date,deputy_id
0,630,678,65100001,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2016-06-15 00:00:00,99111772
1,1476,1479,,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-12-02 00:00:00,99111772
2,2518,2522,,[АБАЛАКОВ А. Н. Поправка предполагает увеличен...,[],Абалаков Александр Николаевич,2015-12-02 00:00:00,99111772
3,4862,4874,65100003,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-11-20 00:00:00,99111772
4,4672,4716,,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-11-18 00:00:00,99111772


In [5]:
data['lines_processed'] = data['lines'].apply(lambda x: re.sub(r'\s+', ' ', ' '.join(x)))
data['lines_processed'] = data['lines_processed'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))
data['lines_processed'] = data['lines_processed'].apply(lambda x: x.strip())
data['lines_processed'] = data.apply(remove_name_from_text, axis=1)

In [6]:
lem = Mystem()
nltk.download('stopwords')
stop = stopwords.words('russian')

def preprocess_text(x):
    return [word.rstrip() for word in (''.join(lem.lemmatize(x))).split(' ') if (word not in stop and word != '')]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitrys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
stop.extend([
u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'наш', u'весь', u'хотя', u'такое', u'например', u'кароч', u'как-то',
u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок',
u'мочь'
])

In [2]:
text_data = data['lines_processed'].to_numpy()

text_data_lemmatized = []
for text in tqdm_notebook(text_data):
    text_data_lemmatized.append(preprocess_text(text))

In [11]:
text_data_lemmatized[0][:10]

['фракция',
 'кпрф',
 'уважаемый',
 'сергей',
 'евгеньевич',
 'уважаемый',
 'товарищ',
 'знать',
 'качество',
 'управление']

In [13]:
data['lines_lemmatized'] = text_data_lemmatized
data.head()

,start_line,end_line,type,lines,votes,deputy_name,meeting_date,deputy_id,lines_processed,lines_lemmatized
0,630,678,65100001,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2016-06-15 00:00:00,99111772,фракция кпрф уважаемый сергей евгеньевич уважа...,"[фракция, кпрф, уважаемый, сергей, евгеньевич,..."
1,1476,1479,,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-12-02 00:00:00,99111772,фракция кпрф прошу вынести на отдельное голосо...,"[фракция, кпрф, просить, вынести, отдельный, г..."
2,2518,2522,,[АБАЛАКОВ А. Н. Поправка предполагает увеличен...,[],Абалаков Александр Николаевич,2015-12-02 00:00:00,99111772,поправка предполагает увеличение взноса на кап...,"[поправка, предполагать, увеличение, взнос, ка..."
3,4862,4874,65100003,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-11-20 00:00:00,99111772,фракция кпрф уважаемый сергей евгеньевич уважа...,"[фракция, кпрф, уважаемый, сергей, евгеньевич,..."
4,4672,4716,,"[АБАЛАКОВ А. Н., фракция КПРФ. ...",[],Абалаков Александр Николаевич,2015-11-18 00:00:00,99111772,фракция кпрф уважаемый сергей евгеньевич уважа...,"[фракция, кпрф, уважаемый, сергей, евгеньевич,..."


In [14]:
data.to_pickle('../data/processed/transcripts_1994_2021_lematized.pkl')

In [15]:
data.shape

(359077, 10)